# Computational Theory Assessment
---

In [89]:
# Import necessary libraries
import numpy as np

## Problem 1: Binary Words and Operations

### Implement the following functions in Python. Use **NumPy** to ensure that all variables and values are treated as **32-bit integers**. These functions are defined in the **Secure Hash Standard (see page 10)**.

1. `Parity(x, y, z)`
2. `Ch(x, y, z)`
3. `Maj(x, y, z)`
4. `Sigma0(x)` - *written as $\Sigma_0^{256}(x)$ in the standard.*
5. `Sigma1(x)` - *written as $\Sigma_1^{256}(x)$ in the standard.*
6. `sigma0(x)` - *written as $\sigma_0^{256}(x)$ in the standard.*
7. `sigma1(x)` - *written as $\sigma_1^{256}(x)$ in the standard.*

### Instructions

- Document each function with a clear docstring
- Explain its purpose and behaviour in Markdown
- Test it with appropriate examples to verify correctness.

### Solution:

**1. Parity(x, y, z) Function (Used in SHA-1 - deprecated)**

The Parity function is a linear bitwise operation used in certain rounds of the SHA-1 hash algorithm (specifically rounds 20-39 and 60-79). It takes three 32-bit words, which we can call `x`, `y`, and `z`, and combines them by applying the XOR operation to each corresponding bit. 

$$
Parity(x, y, z) = x \oplus y \oplus z
$$

XOR (exclusive OR) works like this: for each bit position, if the number of `1`s in that position across `x`, `y`, and `z` is **odd**, the result bit is `1`. If it is **even**, then the result bit is `0`. 

This function helps increase **diffusion** in the hash algorithm, meaning that small changes in input spread out and affect many bits in the output. This contributes to the security of SHA-1 by making it more resistant to certain types of cryptographic attacks.

However, despite these design features, **SHA-1 is no longer considered secure due to successful collision attacks**. A significant milestone was `The SHAppening in 2015`, when researchers demonstrated a practical collision attack on SHA-1 using affordable computing resources. This breakthrough helped accelerate the widespread deprecation of SHA-1.

Today, major organizations have phased out SHA-1 in favor of stronger algorithms like SHA-2 and SHA-3, which do not use the Parity function.

*Resources:*
1. https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.180-4.pdf
2. https://en.wikipedia.org/wiki/SHA-1

In [90]:
def parity(x: int, y: int, z: int) -> np.uint32:
    """
    Performs a bitwise XOR (parity function) across three 32-bit integers.

    This function is used in the SHA-1 algorithm during rounds 20-39 and 60-79.

    Parameters:
        x (int): The first 32-bit integer.
        y (int): The second 32-bit integer.
        z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of x XOR y XOR z, as a 32-bit unsigned integer.
    """

    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return x ^ y ^ z


In [91]:
# Test cases to validate the Parity function
def test_parity():
    test_cases = [
        # Basic tests (decimal)
        ((0, 0, 0), 0), 
        ((1, 0, 0), 1),
        ((1, 1, 0), 0),
        ((1, 1, 1), 1),

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b0000),
        ((0b1100, 0b0011, 0b0101), 0b1010),
        ((0b1111, 0b1111, 0b1111), 0b1111),

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0x00000000, 0x00000000), 0xFFFFFFFF),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0x00000000), 0x00000000),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF), 0xFFFFFFFF),
        ((0xAAAAAAAA, 0x55555555, 0xFFFFFFFF), 0x00000000),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]

    for (x, y, z), expected in test_cases:
        result = parity(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for parity({hex(x)}, {hex(y)}, {hex(z)})"
        )

    # Final complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)
    result = parity(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for parity({hex(x)}, {hex(y)}, {hex(z)})"
    )

    print("All parity function test cases passed.")

test_parity()

All parity function test cases passed.


**2. Ch(x, y, z) Function (Used in SHA-1 and all SHA-2 variants)**

The Ch (Choice) function is a nonlinear bitwise operation used in both SHA-1 (specifically in rounds 0-19) and all SHA-2 variants. It takes three 32-bit (or 64-bit, depending on the SHA-2 variant) words as input: `x`, `y`, and `z`.

$$
Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

The Ch function operates as a bit selector or multiplexer:
- For each bit position, if the bit in `x` is `1`, the function selects the corresponding bit from `y`
- If the bit in `x` is `0`, the function selects the corresponding bit from `z`

This provides important **nonlinear mixing** in the hash algorithm. Unlike the Parity function which is linear, the Ch function has nonlinear behavior that strengthens the cryptographic properties of SHA algorithms by making them more resistant to differential cryptanalysis and other attacks.

Although **SHA-1 is deprecated** due to proven collision vulnerabilities, the `Ch` function remains an essential part of **SHA-2**, which is still considered secure and widely used today.

The name "Ch" reflects its behavior as a "chooser" or "selector" between the bits of `y` and `z` based on `x`.

In [92]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Performs the Choice function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 0-19 
    - All SHA-2 variants
    
    Parameters:
        x (int): Selector 32-bit integer.
        y (int): Bits to be selected when x bit is 1.
        z (int): Bits to be selected when x bit is 0.

    Returns:
        np.uint32: The result of (x & y) ^ (~x & z), as a 32-bit unsigned integer.
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)

    return (x & y) ^ (~x & z)

In [93]:
# Test cases to validate the ch function
def test_ch():
    test_cases = [
        # Basic tests (decimal)
        ((0, 0, 0), 0), 
        ((1, 0, 0), 0),   
        ((1, 1, 0), 1),   
        ((1, 1, 1), 1),   

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b0101),  
        ((0b1100, 0b0011, 0b0101), 0b0001),  
        ((0b1111, 0b1111, 0b1111), 0b1111),  

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0xAAAAAAAA, 0x55555555), 0xAAAAAAAA),
        ((0x00000000, 0xAAAAAAAA, 0x55555555), 0x55555555),
        ((0xAAAAAAAA, 0xFFFFFFFF, 0x00000000), 0xAAAAAAAA),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]
    
    for (x, y, z), expected in test_cases:
        result = ch(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for ch({hex(x)}, {hex(y)}, {hex(z)})"
        )
    
    # Complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = (np.uint32(x) & np.uint32(y)) ^ (~np.uint32(x) & np.uint32(z))
    result = ch(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for ch({hex(x)}, {hex(y)}, {hex(z)})"
    )
    
    print("All ch function test cases passed.")

# Run the tests
test_ch()

All ch function test cases passed.


**3. Maj(x, y, z) Function (Used in SHA-1 and all SHA-2 variants)**

The Maj (Majority) function is a nonlinear bitwise operation used in both SHA-1 (specifically rounds 40-59) and all SHA-2 variants. It takes three 32-bit (or 64-bit, depending on the SHA-2 variant) words as input: `x`, `y`, and `z`.

$$
Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$

The Maj function implements a bit-wise majority vote: for each bit position, the output bit is `1` if at least two of the three input bits are `1`, otherwise it is `0`.

This function provides important **nonlinear mixing** in the hash algorithm. The majority operation effectively contributes to the avalanche effect, where small changes in input bits cause widespread and complex changes in the output. This property is essential for the cryptographic strength of SHA algorithms.

Although **SHA-1 is deprecated** due to collision vulnerabilities, the `Maj` function remains an essential part of **SHA-2**, which is still considered secure and widely used today.

The name "Maj" reflects its behavior as a "majority" selector that chooses the most common bit value among the three inputs at each position.


In [94]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Performs the Majority function on three 32-bit integers.
    
    This function is used in:
    - SHA-1 algorithm during rounds 40-59
    - All SHA-2 variants
    
    Parameters:
        x (int): The first 32-bit integer.
        y (int): The second 32-bit integer.
        z (int): The third 32-bit integer.

    Returns:
        np.uint32: The result of (x & y) ^ (x & z) ^ (y & z), as a 32-bit unsigned integer.
    """
    # Ensure inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (x & z) ^ (y & z)  

In [95]:
# Test cases to validate the Maj function
def test_maj():
    test_cases = [
        # Basic tests
        ((0, 0, 0), 0),
        ((1, 0, 0), 0),
        ((1, 1, 0), 1),
        ((1, 1, 1), 1),

        # Bit pattern tests (binary)
        ((0b1010, 0b0101, 0b1111), 0b1111),
        ((0b1100, 0b0011, 0b0101), 0b0101),
        ((0b1111, 0b1111, 0b1111), 0b1111),

        # Full 32-bit tests (hex)
        ((0xFFFFFFFF, 0x00000000, 0x00000000), 0x00000000),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0x00000000), 0xFFFFFFFF),
        ((0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF), 0xFFFFFFFF),
        ((0xAAAAAAAA, 0x55555555, 0xFFFFFFFF), 0xFFFFFFFF),

        # Edge cases (hex)
        ((0x00000000, 0x00000000, 0x00000000), 0x00000000),
    ]
    
    for (x, y, z), expected in test_cases:
        result = maj(x, y, z)
        assert result == expected, (
            f"Expected {hex(expected)}, got {hex(result)} "
            f"for maj({hex(x)}, {hex(y)}, {hex(z)})"
        )
    
    # Complex value test
    x, y, z = 0x12345678, 0x87654321, 0xDEADBEEF
    expected = (np.uint32(x) & np.uint32(y)) ^ (np.uint32(x) & np.uint32(z)) ^ (np.uint32(y) & np.uint32(z))
    result = maj(x, y, z)
    assert result == expected, (
        f"Expected {hex(expected)}, got {hex(result)} "
        f"for maj({hex(x)}, {hex(y)}, {hex(z)})"
    )
    
    print("All maj function test cases passed.")

# Run the tests
test_maj()

All maj function test cases passed.


**4. Sigma0(x) Function (Used in SHA-256 and SHA-224)**

The `Sigma0(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It takes a single 32-bit word `x` as input.

$$
\Sigma_0^{256}(x) = ROTR^2(x) \oplus ROTR^{13}(x) \oplus ROTR^{22}(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions.

The function takes the input 32-bit word `x` and performs three different circular right rotations (by 2, 13, and 22 bits), then combines the results using the XOR operation. 

This operation provides important **nonlinear diffusion** in the hash algorithm. The combination of different rotation amounts ensures that changes in the input bits propagate widely and unpredictably throughout the output, contributing to the cryptographic strength of SHA-256 and SHA-224.


In [96]:
def big_sigma0(x: int) -> np.uint32:
    """
    Performs the Σ₀ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function 
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input 
    word by 2, 13, and 22 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x), as a 32-bit unsigned integer.
    """
    x = np.uint32(x)

    # Perform the circular right rotations 
    rotr2 = np.uint32((x >> 2) | (x << 30)) # 32 - 2 = 30
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 32 - 13 = 19
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 32 - 22 = 10

    return rotr2 ^ rotr13 ^ rotr22


In [97]:
# Test cases to validate the big_sigma0 function
def test_big_sigma0():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0xffffffff), 
        (0x80000000, 0x20040200), 
        (0x12345678, 0x66146474), 
    ]

    for x, expected in test_cases:
        result = big_sigma0(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All big_sigma0 test cases passed.")

# Run the test
test_big_sigma0()

All big_sigma0 test cases passed.


**5. Sigma1(x) Function (Used in SHA-256 and SHA-224)**

The `Sigma1(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It takes a single 32-bit word `x` as input.

$$
\Sigma_1^{256}(x) = ROTR^6(x) \oplus ROTR^{11}(x) \oplus ROTR^{25}(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions.

The function takes the input 32-bit word `x` and performs three different circular right rotations (by 6, 11, and 25 bits), then combines the results using the XOR operation.

This operation provides important **nonlinear diffusion** in the hash algorithm. The combination of different rotation amounts ensures that changes in the input bits propagate widely and unpredictably throughout the output, contributing to the cryptographic strength of SHA-256 and SHA-224.


In [98]:
def big_sigma1(x: int) -> np.uint32:
    """
    Performs the Σ₁ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input
    word by 6, 11, and 25 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x), as a 32-bit unsigned integer.
    """
    x = np.uint32(x)

    # Perform the circular right rotations
    rotr6 = np.uint32((x >> 6) | (x << 26))  # 32 - 6 = 26
    rotr11 = np.uint32((x >> 11) | (x << 21))  # 32 - 11 = 21
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 32 - 25 = 7

    return rotr6 ^ rotr11 ^ rotr25

In [99]:
# Test cases to validate the big_sigma1 function
def test_big_sigma1():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0xffffffff), 
        (0x80000000, 0x2100040), 
        (0x12345678, 0x3561abda), 
    ]

    for x, expected in test_cases:
        result = big_sigma1(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All big_sigma1 test cases passed.")

# Run the test
test_big_sigma1()

All big_sigma1 test cases passed.


**6. sigma0(x) Function (Used in SHA-256 and SHA-224)**

The `sigma0(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It operates on a single 32-bit word and is used in the message schedule.

$$
\sigma_0^{256}(x) = ROTR^7(x) \oplus ROTR^{18}(x) \oplus SHR^3(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions, and `SHR^n(x)` represents a right shift of `x` by `n` positions (filling with zeros on the left).

The function takes the input 32-bit word `x` and performs two circular right rotations (by 7 and 18 bits) and one right shift (by 3 bits), then combines the results using the XOR operation.

This function is used in the message schedule expansion (section 6.2.2 of the Secure Hash Standard) in SHA-256 and SHA-224. It provides crucial **nonlinear diffusion** in the algorithm. The combination of rotations and shifts ensures that bit changes propagate throughout the word in complex patterns, contributing to the cryptographic strength of SHA-256.


In [100]:
def small_sigma0(x: int) -> np.uint32:
    """
    Performs the σ₀ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 7 and 18 positions, one right shift (SHR) by 3 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x), as a 32-bit unsigned integer
    """
    # Ensure input is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Perform circular right rotations by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25)) # 32 - 7 = 25
    rotr18 = np.uint32((x >> 18) | (x << 14)) # 32 - 18 = 14

    # Perform right shift by 3 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr3 = np.uint32(x >> 3)
    
    # XOR all three values together
    return rotr7 ^ rotr18 ^ shr3


In [101]:
# Test cases to validate the small_sigma0 function
def test_small_sigma0():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0x1fffffff), 
        (0x80000000, 0x11002000), 
        (0x12345678, 0xe7fce6ee), 
    ]

    for x, expected in test_cases:
        result = small_sigma0(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All small_sigma0 test cases passed.")

# Run the test
test_small_sigma0()

All small_sigma0 test cases passed.


**7. sigma1(x) Function (Used in SHA-256 and SHA-224**

The `sigma1(x)` function is a nonlinear bitwise operation used in the SHA-256 and SHA-224 hash algorithms. It operates on a single 32-bit word and is used in the message schedule.

$$
\sigma_1^{256}(x) = ROTR^{17}(x) \oplus ROTR^{19}(x) \oplus SHR^{10}(x)
$$

Where `ROTR^n(x)` represents a circular right rotation of the bits in `x` by `n` positions, and `SHR^n(x)` represents a right shift of `x` by `n` positions (filling with zeros on the left).

The function takes the input 32-bit word `x` and performs two circular right rotations (by 17 and 19 bits) and one right shift (by 10 bits), then combines the results using the XOR operation.

This function is used in the message schedule expansion (section 6.2.2 of the Secure Hash Standard) in SHA-256 and SHA-224. It provides crucial nonlinear diffusion in the algorithm. The combination of rotations and shifts ensures that bit changes propagate throughout the word in complex patterns, contributing to the cryptographic strength of SHA-256.


In [102]:
def small_sigma1(x: int) -> np.uint32:
    """
    Performs the σ₁ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 17 and 19 positions, one right shift (SHR) by 10 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x), as a 32-bit unsigned integer
    """
    # Ensure input is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Perform circular right rotations by 17 and 19 bits
    rotr17 = np.uint32((x >> 17) | (x << 15)) # 32 - 17 = 15
    rotr19 = np.uint32((x >> 19) | (x << 13)) # 32 - 19 = 13

    # Perform right shift by 10 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr10 = np.uint32(x >> 10)
    
    # XOR all three values together
    return rotr17 ^ rotr19 ^ shr10

In [103]:
# Test cases to validate the small_sigma1 function
def test_small_sigma1():
  
    test_cases = [
        (0x00000000, 0x00000000), 
        (0xFFFFFFFF, 0x003fffff), 
        (0x80000000, 0x205000), 
        (0x12345678, 0xa1f78649), 
    ]

    for x, expected in test_cases:
        result = small_sigma1(x)
        assert result == expected, (
            f"Test failed for input {hex(x)}: "
            f"expected {hex(expected)}, got {hex(result)}"
        )

    print("All small_sigma1 test cases passed.")

# Run the test
test_small_sigma1()

All small_sigma1 test cases passed.


---

## Problem 2: Fractional Parts of Cube Roots

### Use **NumPy** to calculate the constants listed at the bottom of **page 11 of the Secure Hash Standard**, following the steps below. These are the **first 32 bits of the fractional parts** of the **cube roots of the first 64 prime numbers**.

### Instructions

- Write a function called `primes(n)` that generates the first `n` prime numbers.
- Use the function to calculate the **cube root** of the first 64 primes.
- For each cube root, extract the first thirty-two bits of the fractional part.
- Display the result in **hexadecimal**.
- Test the results against what is in the Secure Hash Standard.

### Solution:

In the Secure Hash Standard, a sequence of 64 constant values is used during the compression function. These constants are derived from the **first 32 bits of the fractional parts of the cube roots** of the first 64 prime numbers (as defined in section 4.2.2 of the standard).

This problem walks through how to compute these constants programmatically using NumPy, and verify that they match the official values provided by NIST.

We begin by implementing the `primes(n)` function, which generates the first `n` prime numbers.

For this, I use a basic trial division approach, which is efficient for small values of `n` (such as the first 64 primes). For significantly larger values of `n`, a more advanced algorithm like the **Sieve of Eratosthenes** would be more appropriate.

This implementation handles **edge cases** such as `n ≤ 0`, and includes a **performance optimization:** it skips all even numbers after 2. Additionally, it only checks for divisibility using previously found primes, up to the square root of the candidate number, reducing unnecessary computations.

In [104]:
def primes(n):
    """
    Generates the first n prime numbers using a simple trial division method.
    
    Parameters:
        n (int): The number of prime numbers to generate
        
    Returns:
        list: A list containing the first n prime numbers
    """
    if n <= 0:
        return [] # Return an empty list for non-positive input

    prime_list = [2] # List to store prime numbers starting with the first prime
    candidate = 3 # Start checking for primes from the first odd prime number

    # While loop until we have n primes
    while len(prime_list) < n:
        is_prime = True # Assume candidate is prime until proven otherwise
        for p in prime_list: # Check divisibility with known primes
            if p * p > candidate: 
                break # No need to check further if p^2 exceeds candidate
            if candidate % p == 0:
                is_prime = False # Found a divisor, not prime
                break
        
        if is_prime:
            prime_list.append(candidate) # Add candidate to the list if it's prime
        
        candidate += 2 # Odd numbers only, skip even numbers

    return prime_list # Return the list of prime numbers

Next, we generate the **first 64 prime numbers** using the `primes(n)` function.

In [105]:
# Generate the first 64 prime numbers
first_64_primes = primes(64)

For each prime number, we calculate the **cube root**, extract its **fractional part**, and multiply it by $2^{32}$ to obtain the first **32 bits** of the fractional part.

These 32-bit values correspond to the **SHA-256 constants** defined in the Secure Hash Standard.

In [106]:
sha256_constants = [] # List to store the K constants

for prime in first_64_primes: 
    cube_root = np.cbrt(prime) # Calculate the cube root
    fractional_part = cube_root - np.floor(cube_root) # Extract the fractional part
    bits32 = np.uint32(fractional_part * (2**32)) # Scale to 32 bits
    sha256_constants.append(bits32) # Append to the constants list

Finally, we display the constants in **hexadecimal format**, grouped in sets of 8, as shown in the *Secure Hash Standard*.

In [107]:
# Display constants in hexadecimal format (8 per line)
for i in range(0, 64, 8):
    print(', '.join(f"0x{k:08x}" for k in sha256_constants[i:i+8])) # Display in hex format

0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5
0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174
0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da
0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967
0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85
0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070
0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3
0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2


The following test verifies that the calculated SHA-256 K constants exactly match the official constants listed in the Secure Hash Standard documentation.  
Passing this test ensures the correctness of the constants used in our SHA-256 implementation.

In [108]:
def test_sha256_constants():
    """
    Test that the SHA-256 K constants match the official constants
    as specified in the SHA-256 standard.
    """
    official_constants = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]

    assert sha256_constants == official_constants, "SHA-256 K constants do not match official values"
    print("Test passed: SHA-256 K constants match the official values.")

# Run the test
test_sha256_constants()

Test passed: SHA-256 K constants match the official values.


---

## Problem 3: Padding

### Instructions

- Write a generator function `block_parse(msg)` that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 
- The function should accept a bytes object called `msg`. 
- At each iteration, it should yield the next 512-bit block of `msg` as a bytes object. 
- Ensure that the final block (or final two blocks) include the required padding of `msg` as specified in the standard. 
- Test the generator with messages of different lengths to confirm proper padding and block output.

### Solution:

SHA algorithms operate on fixed-size blocks of data (512 bits for SHA-256). Before hashing, messages must be padded according to rules defined in sections 5.1.1 and 5.2.1 of the Secure Hash Standard. This ensures the final message length is compatible with the algorithm’s block size requirements.

The goal of this problem is to implement a generator function, `block_parse(msg)`, that yields successive 512-bit (64-byte) blocks from the input message, correctly applying the necessary padding. This includes appending a '1' bit, enough zeros, and the original message length as a 64-bit integer, ensuring the padded message is a multiple of 512 bits.

This block-wise processing is critical for the SHA hashing process. Without proper padding, the message would not align with the internal structure of the SHA-256 compression function, potentially leading to incorrect hash outputs.

In [ ]:
def block_parse(msg):
    """
    Generator function that parses a message into 512-bit blocks with SHA-256 padding.
    
    Implements padding according to sections 5.1.1 and 5.2.1 of the Secure Hash Standard.
    
    Parameters:
        msg (bytes): The input message to be parsed
        
    Yields:
        bytes: Each 512-bit (64-byte) block of the padded message
    """
    # Length of original message in bits
    msg_len_bits = len(msg) * 8
    
    # Convert message to mutable bytearray for padding 
    padded_msg = bytearray(msg)
    
    # Append the '1' bit (10000000 in binary, or 0x80 in hex)
    padded_msg.append(0x80)
    
    # Calculate the length mod 64 to determine padding needed to reach 56 bytes (448 bits)
    padding_mod = len(padded_msg) % 64
    padding_needed = (56 - padding_mod) if padding_mod <= 56 else (120 - padding_mod)
    
    # Append zero bytes to reach the required length before length encoding
    padded_msg.extend([0x00] * padding_needed)
    
    # Append the original message length as a 64-bit big-endian integer
    padded_msg.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    # Yield successive 512-bit (64-byte) blocks
    for i in range(0, len(padded_msg), 64):
        yield bytes(padded_msg[i:i+64])


---

## Problem 4: Hashes

### Instructions

- Write a function `hash(current, block)` that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

### Solution:

---

## Problem 5: Passwords

### Instructions

- The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. 
- As strings, they were encoded using UTF-8. 
- Determine the passwords and explain how you found them. 
- Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

### SHA-256 hashes 
1. 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
2. 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
3. b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342

### Solution:

---
# End of Assessment